In [1]:
import pandas as pd
import numpy as np

object_key_train = 'data/external_parties_train.csv'
object_key_test = 'data/external_parties_test.csv'


df_train = pd.read_csv(object_key_train)
infos_train = df_train['party_info_unstructured']
ids_train = df_train['transaction_reference_id']

df_test = pd.read_csv(object_key_test)
infos_test = df_test['party_info_unstructured']
ids_test = df_test['transaction_reference_id']


print(ids_test)
print("-------")
print(ids_train)


0          8f71ed0a819236b141978defe9a98700
1          dc9be336b81b971c04ca98ccdf99d51e
2          29c8ed895b8a220121168158a7447773
3          38ee5d04ad50ddf5f6abc67017201548
4          eef4f720e71bda0a6f1e7caa21a2c98c
                         ...               
1481667    a3b0502a9917c0da0c0343055d434c8f
1481668    38aac541d4b2cef7c05b584abfcca494
1481669    28f3dfc3160bae3b92c7c42e1082cf43
1481670    39f8569b0ced6d11fad95d628dd05d6a
1481671    ca917bf58677e7cd34b21d38a19ee17c
Name: transaction_reference_id, Length: 1481672, dtype: object
-------
0        04ff0d1c680189e3a80c92d86407f0f5
1        439ab0ad7380e6135ab2ff3fddd4a727
2        00cac12d41191a84f9e31aa731a83512
3        e4fba5f878dd3453e35973605a783a16
4        d03d7e4c31878b0255d39e8c3f0ab625
                       ...               
11059    7183d9c3700148c9527869948b685085
11060    b47b9ed0a8cc9fcc4c21ac368fe79757
11061    d61ff2b0184f32ad0021a313c6112b2e
11062    bec335b1b1bad8c55b7dce549cfd8de0
11063    c2a8edf088268bdd

In [2]:
import pickle

In [15]:
# Load the encoded data
with open('cluster_labels_PartialEncoder.pkl', 'rb') as f:
    cluster_labelsPartialEncoder = pickle.load(f)

print("Loaded encoded test data:")
print(cluster_labelsPartialEncoder.shape)

Loaded encoded test data:
(900000,)


In [16]:
list_labelsPartialEncoder = cluster_labelsPartialEncoder.tolist()

# append unique values to the list to make it the same length as the test data
curr = max(list_labelsPartialEncoder) + 1
len_list = len(list_labelsPartialEncoder)

for i in range(len_list, len(infos_test)):
    list_labelsPartialEncoder.append(curr)
    curr += 1
    
print(len(list_labelsPartialEncoder))
print(len(infos_test))

1481672
1481672


In [17]:
# Create a dictionary to map each unique IBAN to a unique cluster label
iban_to_cluster = {iban: idx for idx, iban in enumerate(df_test['party_iban'].unique())}
iban_to_cluster[np.nan] = -1

# Assign cluster labels to each row based on the IBAN
df_test['external_id'] = df_test['party_iban'].map(iban_to_cluster)

# Extract the cluster labels as an array
cluster_labels = df_test['external_id'].values
list_labelsIBAN = cluster_labels.tolist()


In [19]:
#modify the cluster labels in the following way: if the cluster label is -1, we assign a unique label of the form -n with n natural number
count = len(df_test['party_iban'].unique())

for i in range(len(df_test['external_id'])):
    if list_labelsIBAN[i] == -1:
        list_labelsIBAN[i] = count
        count += 1

In [18]:
from networkx.utils import UnionFind
from collections import defaultdict

def merge_clusterings(clustering1, clustering2):
    uf = UnionFind()

    dic1 = defaultdict(list)
    dic2 = defaultdict(list)
    for idx, label in enumerate(clustering1):
        dic1[label].append(idx)
    for idx, label in enumerate(clustering2):
        dic2[label].append(idx)
    
    # dic1 = {label1: {idx1, idx2, ...}, label2: {idx3, idx4, ...}, ...}

    for label, indices in dic1.items():
        for idx in indices[1:]:
            uf.union(indices[0], idx)
    for label, indices in dic2.items():
        for idx in indices[1:]:
            uf.union(indices[0], idx)

    return [uf[idx] for idx in range(len(clustering1))]

In [ ]:
#merge the two clusterings 
list_labelsPE_IBAN = merge_clusterings(list_labelsPartialEncoder, list_labelsIBAN)
